In [21]:
import pandas as pd
import requests
import json
import time

In [22]:
station_df = pd.read_csv('data/avg_rent_price_with_latlon.csv')


# 一都三県のみ残す (11, 12 ,13, 14)
station_df = station_df[station_df['pref_cd'] >= 11]
station_df = station_df[station_df['pref_cd'] <= 14]

tt_df = pd.DataFrame(station_df['station_name'])
tt_df['travel_time'] = 0

for name in tt_df['station_name']:
    response = requests.get('https://api.ekispert.jp/v1/json/search/course/extreme?key=test_b5vxpCyab7d&viaList=渋谷:'+name+'&conditionDetail=T11211332313191:F3321122120000:A23121141:')
    if response.status_code != 200:
        if response.status_code == 400:
            print(name)
            print(response.status_code)
            print(response.text)
        else:
            print("Unknown error")
            print(name)
            print(response.status_code)
            print(response.text)
        next
        
    print(name + ": OK")
    # リクエスト間隔調整
    time.sleep(0.7)

    
    

東京: OK
有楽町: OK
新橋: OK
浜松町: OK
田町
400
{"ResultSet":{"apiVersion":"1.27.0.0","engineVersion":"202112_05a","Error":{"code":"E102","Message":"駅名が見つかりません。(田町)"}}}
田町: OK
高輪ゲートウェイ: OK
品川: OK
大崎: OK
五反田: OK
目黒: OK
恵比寿: OK
渋谷: OK
原宿: OK
代々木: OK
新宿: OK
新大久保: OK
高田馬場: OK
目白: OK
池袋: OK
大塚
400
{"ResultSet":{"apiVersion":"1.27.0.0","engineVersion":"202112_05a","Error":{"code":"E102","Message":"駅名が見つかりません。(大塚)"}}}
大塚: OK
巣鴨: OK
駒込: OK
田端: OK
西日暮里: OK
日暮里: OK
鶯谷: OK
上野: OK
御徒町: OK
秋葉原: OK
神田
400
{"ResultSet":{"apiVersion":"1.27.0.0","engineVersion":"202112_05a","Error":{"code":"E102","Message":"駅名が見つかりません。(神田)"}}}
神田: OK
大宮
400
{"ResultSet":{"apiVersion":"1.27.0.0","engineVersion":"202112_05a","Error":{"code":"E102","Message":"駅名が見つかりません。(大宮)"}}}
大宮: OK
さいたま新都心: OK
与野: OK
北浦和: OK
浦和: OK
南浦和: OK
蕨: OK
西川口: OK
川口: OK
赤羽: OK
東十条: OK
王子: OK
上中里: OK
大井町: OK
大森
400
{"ResultSet":{"apiVersion":"1.27.0.0","engineVersion":"202112_05a","Error":{"code":"E102","Message":"駅名が見つかりません。(大森)"}}}
大森: OK
蒲田: OK
川崎: OK
鶴

KeyboardInterrupt: 